In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
print("This is release 2")

In [ ]:
from databricks.connect import DatabricksSession
spark = DatabricksSession.builder.getOrCreate()

In [ ]:
%pip install openmeteo-requests==1.2.0
%pip install requests-cache==1.2.0
%pip install retry-requests==2.0.0
%pip install geopy==2.4.1
dbutils.library.restartPython()

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [ ]:
import pandas as pd
from weather_utilities.coffee_weather import find_region_latlong, pull_weather_data

In [ ]:
catalog_name = 'shirlywang_sandbox'

In [ ]:
dbutils.widgets.text("schema_name", "")
schema_name = dbutils.widgets.get("schema_name")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}")

In [29]:
coffee_data = spark.read.table("dnb_wave_5.sandbox.coffee_raw")

In [4]:
unique_regions = coffee_data.select("Country of Origin").distinct().collect()
region_list = [row["Country of Origin"] for row in unique_regions]

In [21]:
geocodes = find_region_latlong(region_list)
latitude = [loc.latitude for location_dict in geocodes for country, loc in location_dict.items()]
longitude = [loc.longitude for location_dict in geocodes for country, loc in location_dict.items()]

location_weather_data = pull_weather_data(latitude, longitude)

In [25]:
df = pd.DataFrame(location_weather_data)
df['date'] = df['date'].apply(lambda x: [date.date() for date in x])

spark_df = spark.createDataFrame(df)

In [26]:
extracted_info = [{"country": country, "latitude": loc.latitude, "longitude": loc.longitude} for location_dict in geocodes for country, loc in location_dict.items()]
region_location_map = spark.createDataFrame(extracted_info)

In [27]:
joined = spark_df \
    .join(region_location_map, spark_df.latitude == region_location_map.latitude, how='left') \
    .drop(region_location_map.latitude, region_location_map.longitude)

In [30]:
joined.write.mode("ignore").format("delta").saveAsTable(f"{catalog_name}.{schema_name}.location_weather_raw")

In [ ]:
print(1)